## The Engine of neural networks: Gradient based optimization
---

Each layer in neural network transform its data as follows:
`output = relu(dot(w,input)+b)`  

In this expression, w and bare tensors that are attributes of the layer.They're called the weights or trainable parameters of the layer(the kernel and bias attributes, respectively).These weights containth information learned by the network from exposure to trianing data.  

Initally, these weight matrices are filled with small random values(a step called random initialization).Of course, ther's no reason to expcept that `relu(dot(w,input)+b)`, where w and b are random, will yield useful representation.The resulting representation are meaningless but they are the starting point.What comes next is to gradually adjust these weights, base on a feedback signal.This gradual adjustment, also called training, is basically the learning that machine learning is all about.   

This happens within waht's called a *training loop*, which works as follows, Repeat these steps in a loop as long as necessary:  

1. Draw batch of traiang samples x and crossesponding targets y.
2. Run the networkk on x(a step called forward pass) to obtain the prediction `y_pred`.
3. Compute the loass of the network on the batch, a measure of the mismatch between `y_pred` and `y`.
4. Update all weights on the network in a way that slightly reduces the loss on this batch.


You'll evantually end up with the netwok that has  a very low loss on its training data: a low mismatch between prediction y_pred and expeted targets y.The networks has 'learned'to mao its inouts to correct targets.From afar, it may look like magix, ut when you reduce it to elementry steps, it turns out to be simple.  

Step 1 sounds easy enough  just I/O code.Step 2 and 3 are mearely the application of a handful of tensor operaitons. The difficult part is step 4: updating the network's weights.Given a individual weights coeffiecent in the network, how can you compute whether the coefficient should be increased or decreased, and by how much.  

One naive solution would be to freeze alll weights in the network except the one scalar coeffiecient being cosidered and try different values for this coeffiecents.Let's say the initial value of the coefficinet is 0.3.After the forward pass on the batch of the data, the loss of the network on the batch is 0.5.If uou change the coefficint's value to 0.35 and rerun the forward pass, the loss increase to 0.6.But if you lower the coeffiencts to 0,25, the loss falls to 0.4.In this case. it seems that updating the coeffeinct by -0.05 would contribute to minmiizng the loss.THis would have to be repeated for all coefficient in the network.  

But such an approcah would be hottinlu inefficient, because you'd need to compute two forward pass for every individual coefficient.A much better approach is to take advantage of the fact that all operation used in the network are `differentiable`, and compute the `gradient` of the loss with regard to network's cofficients. You can then move the coefficients in the the opposite direction from th gradient, thus decreasing the loss.

#### What's a derivative?
---

Consider a continous smoth function `f(x) = y`, mapping a real number x to a new real number `y`.Beacuse the function is continous , a small change in x can only result in a small change in y- that's the intuitation behind the contunity.Let's say you increase x by a small factor `epsilon_x`: this result in samm `epsilon_y` change to y.
`f(x+ epsilon_x) = y + epsilon_y`  

In addition, beacuse the function is `smooth`(its curve doesn't have any abrupt angles), when `epsilon_x` is small enough, around a certian point p, it ;s posssible to approximate `f` as a linear function of slope `a`, so that `epsilon_y` becomes `a*epsilon_x`.Obviously, this linear approximation is valid only when `x`is close enough to `p`.  

 The slope `a` is called the `drivative of f in p`. If `a` is negative, it means a small change of `x` around `p` will result in a decrease of `f(x)`, and if `a` is positve, a small change in x will result in an increase of f(x).The absolut value of `a`(magnitude) tells how qucikly this increase or decrease will happen.  
 
For every differentiable function `f(X)` (differentiable means "can be derived": for example, smooth and continous function), there exist a derivative function `f'(x)` that map value of x to the slope of the local linear approximation of f in those points. For instance, the derivative of `cos(x) is -sin(x)`, the derivative of f(x) = a * x is f'(x)=a and so on.  

If you're trying to update `x` by a factor `epsilon_x` in order to minimize `f(x)`, and you know the derivative of `f`, then your job is done: the derivative completely describes how `f(x)` evolves as you change `x`. If we want to reduce the value of `f(x)`, tou just need to move x a little in the opposite direction from the derivative.

#### Derivative of a tesnor operations: the Gradient
---

A `gradient` is the derivative of a tensor operations. It's the generlization of the concept of derivatives to function of multidimensional inputs: that is, to functions that take tensors as inputs.  

Consider an input vector `x`, a matrix `w`, a target `y`, and a loss function `loss`.We can use `w` to compute a traget predictions `y_pred`, and compute the loss, or mismatch, between the target candidate `y_pred` and the target `y`:  

`y_pred = dot(w,x)
loss_value = loss(y_pred,y)` 

If the data inputs `x` and `y` are frozen, then this can be interpreted as a function mapping values of `w` to loss values.  

`loss_value = f(w)`  

Let's say the current value of `w` is `w0`.Then derivative of `f` in the point `w0` is a tensor `gradient(f)(w0)` with the same shape as `w`, where each coeffieicent `gradient(f)(w0)[i,j]` indicates the direction and magnitude of the change in `loss_value` you observe when modifying `w0[i,j]`.The tensor `gradient(f)(w0)` is the gradient of the function `f(w) = loss_value in w0`


We saw earlier that the derivative of a function `f(x)` of a single coefficient can be interpretted as the slope of the curve of `f`.Likewise the gradient(f)(w0) can be interpretted as the tensor describing the curvature of `f(w)` around `w0`  

For this same, we can reduce the value of `f(x)` by moving `x` a little in the opposite directionfrom the derivative, with a function `f(w)` of a tensor, you can reduce `f(w)` by moving `w` in the opposite direction direction from the gradient. for example, `w1 = w0 - step*gradient(f)(w0)` where step is called scaling factor.Thatmeans going against the curvature, which should put upu lower on the curve.   

Note: Scaling factor `step` is needed because `gradient(f)(w0)` only approximates the curvature when you're close to `w0`, so you don't go far from w0.

#### Stochastic gradient descent
---

Given a differntiable function, it is theoretically possible to find its minium analytically, it is known that a function's minimum is a point where the derivative is 0, so all we have to do is finad all the points where the derivative goes to 0 and check for which of these points the function has lowest value.  

Applied to neural network, that means finding analytically the combination of weight values that yields the smallest possible loss functions.This can be done by solving the equation `gradient(f)(w) = 0` for `w`.This is a ploynimal eqaution of `N` varible, where `N` is the coeffiecnts in the network.Although it would be possible to solve such an eqation for N=2 or N=3, doing so id intractable for real neural network, where the number of paramters is never less than a few thousand and often several tens of miilons.  

>Instead, you can use the four step algorithm where modify the parameters little by little based on the current loss value on a random batch data.Because we are dealing with a diffrential equation, we can compute its gradient, If we updates the weights in the opposite direction from the gradient, the loss will be little less every time.  

>>1. Draw batch of training samples X and corresponding target y.
>>2. Run the network on x to obtain prediction y_pred.
>>3. compute the loss of the network on the batch, a measure of the mismatch between y_pred and y.
>>4. compute the gradient of the loss with regard to the network's parameter.
>>5. Move the parameter a little in the opposite direction from the gradient - for example `W -= step * gradient`- thus reducing the loss on the batch a bit.  

What we have just used is `Mini batch stochastic Gradient descent`The term stochastic refers to the fact that each batch of data is drawn at random(stochastic is a scientific synonym of random).  

It is very intuitive and is important to pick a reasonable value for the `step` factor.If it's toll small, the descent down the curve will take many iteration and it could get stuck in local minimum.If step is too large, your updates end up taking you to complety random places.  

Variant of the mini-batch SGD algorithm would be to draw a single sample and target at each iteration, rahther than drawing a batch of data.This would be *true* SGD. Alternatively, going to the opposite extreme, you could run every steps on all data avaailable, which is called SGD.Each update thenbe more accurate, but far more expensive. The effieicent compromise between these two extremes is to use mini-batch of resonable size.  

In practice  we use gradient descent in highly dimensional spaces: every weight coefficients in a neural network is s free dimensional space, and there may be tens of thousands or even million sof them.TO build of innution of loss surface you can visualize it by gradient descent along a 2D surface. We can't possibily visualize the actual process of training of neural netwworls look like- we cannot represent a 1,000,000 dimensional space in a way that makes sense to humnas.It is good to keep in mind that the intuitions you develop through these low-dimensional representations may not always be acccurate in practice.This has historically been a sourceof issues in the world of deep learning research.  

There are multiple variants of SGD exist that differ by taking into **account previous weight updates when computing the next weight updates**, rather the just loking at the current value of the gradients.SGD with  momentum,Adagrad,RMSProp andseveral others. these variants are known as `optimization methods or optimizers`.Momentum used in optimizers addresses with SGD: convergence speed and local minima.As you can see fromthe picture below around certain parameter value, there is local minimum, around that point, moving left will reuslt in loss increasing and so would moving right.If the parameter under consideration is being optiized via SGD with a small learning rate, then the optimization process would get stuck at the local minium instead of making its way to the global minimum.  

We can avoid such issues with momemutum,which is inspiredfrom physics.A useful mental image here is to think of the optimization process as a small ball rolling down the loss curve,If it has enough momentum, the ball wont't get stuck in a ravine and will end up at the global minima.Momentum is impleneted by moving the ball at each step based not only on the current slope value(current accelartion) but allso on the current velcoity(result of past accelaration).In practice this means updating the parameter `w` based not only on the current gradient value but also on th previous parameter update.  

~~~python
past_velocity = 0
momentum = 0.1
while loss > 0.01:
    w,loss,gradient = get_current_parameters()
    velocity = past_velocity*momentum + learning_rate*gradient
    w = w + momentum * velocity - learning_rate*gradient
    past_velocity = velocity
    update_parmaeter(w)
      
    



#### Chaining derivatives: The backpropagation algorithm
---

In neural network because the function is differentiable, we can explicitly compute its derivative.In practice, a neural network function consist of many tensor operations chained together, each of which has a simple, know derivative. For example, let's take a network `f` composed of three tensor operations `a,b and c` with weight matrices `w1,w2 and w3`:  
`f(w1,w2,w3) = a(w1, b(w2,c(w3))`  

Calculus tells us that such a chain of functions can be derived using the folloing identity, called chain rule:  `f(g(x)) = f'(g(x)) * g'(x)`.Applying chain rule to the computations of the gradients value of a neural network give rise to an algorithm called `backpropagation`(also called reverse-mode differentiation).Backpropagation starts with the final loss value and works backward from the top layers to the bottom layers, applying chain rule to compute the contribution that each each parameter had in the loss value.  

Nowadays, people will implement networks in modern frameworkks that are capable of `symbolic differentiation`, such as tesnorflow.This means that, given a chain of operations with a known derivtive, they can compute a gradient function for the chain(by applying the chain rule) that map network parameter values to gradient values.When you have access to such a function.Thanks to symbolic differentiation will never have to implement the backpropagation algorithm by hand.

#### Some Jupyter Text formatting

1. **Task list**

- [ ] prepration of manuscript
- [ ] study graph learning
- [x] implementation of pytorch library  
---

2. **Refrence links**

[python][1] is a programming language, this is mainly used in the industry of machine learning and its various [applications][2].

[1]: <https://www.python.org/> "python software foundation"
[2]: <https://jupyter.orf> "Project Jupyter"
---

3. **Insert Emoji**  

For emoji standard markdown doesn't support that for this copy emoji source code from [emojipidea](https://emojipedia.org/).
- love ❤️
- new year 🎉
---